# Test BSM model implementation
Repeat the same for normal model implementation

In [1]:
import numpy as np
import scipy.stats as ss

In [2]:
# import
from option_models import basket
from option_models import bsm
from option_models import normal

In [3]:
### only run this when you changed the class definition
import imp
imp.reload(bsm)
imp.reload(normal)

<module 'option_models.normal' from 'C:\\Users\\rapha\\OneDrive\\Documents\\GitHub\\PHBS_ASP_2018\\HW3\\option_models\\normal.py'>

### Price

In [3]:
# create model
bsm1 = bsm.BsmModel(0.2)

In [5]:
# price
strike = 102
spot = 100
texp = 0.25

price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
print(price)
assert( abs(price - 3.10628366655) < 1e-10 )

3.1062836665495652


### Implied vol

In [7]:
# Randomly generate spot/strike/expiry/intr/divr/cp_sign
# Then test implied volatility

for k in range(100):
    spot = np.random.uniform(80,100)
    strike = np.random.uniform(80,100)
    vol = np.random.uniform(0.0001, 0.4)
    texp = np.random.uniform(0.1, 5)
    intr = np.random.uniform(0, 0.3)
    divr = np.random.uniform(0, 0.3)
    cp_sign = 1 if np.random.rand() > 0.5 else -1

    #print( spot, strike, vol, texp, intr, divr, cp_sign)

    bsm2 = bsm.BsmModel(vol=vol, intr=intr, divr=divr)
    price = bsm2.price(strike, spot, texp, cp_sign )
    
    # get implied vol
    vol_imp = bsm2.impvol(price, strike, spot, texp=texp, cp_sign=cp_sign)
    
    # now price option with the obtained implied vol
    bsm2.vol = vol_imp
    price_imp = bsm2.price(strike, spot, texp, cp_sign )
    
    # compare the two prices
    assert( abs(price - price_imp) < 1e-8 )

### Delta
Verify delta by comparing numerical derivative

In [22]:
bsm1 = bsm.BsmModel(0.2)
strike = 102
spot = 100
texp = 0.25
h=1e-6

In [11]:
delta = bsm1.delta(strike=strike, spot=spot, texp=texp, cp_sign=1)
price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price_dn = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
delta_numeric = ( price_up - price_dn )/(2*h)
print(delta, delta_numeric)

0.4411610169119095 0.4411610170507174


### Vega
Do similar verification

In [27]:
vol=0.20
vega = bsm1.vega(strike, spot, texp, cp_sign=1)

price_up = bsm.bsm_price(strike=strike, spot=spot, vol=vol+h, texp=texp, cp_sign=1)
price_dn = bsm.bsm_price(strike=strike, spot=spot, vol=vol-h, texp=texp, cp_sign=1)
vega_numeric = ( price_up - price_dn )/(2*h)

print(vega, vega_numeric)

19.72976843913797 19.72976843944707


### Gamma
Do similar verification

In [33]:
gamma = bsm1.gamma(strike, spot, texp, cp_sign=1)

price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
price_dn = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
gamma_numeric = ( price_up - 2*price + price_dn )/(h**2)

print(gamma, gamma_numeric)

0.03945953687827594 0.04973799150320701
